# End Goals
- Proper alignment with other teams, in particular:
    - data scientists
    - product side: Ultimately, the ML system should provide value to the business, and it is the job of the product side to provide this guidance. How to achieve given requirements is the expertise of the ML and data science side. However, it is often not possible to clearly separate these questions, requiring a broader understanding on both sides.
- Manage the complexity of the overall system to ensure:
  - continued agility: Stay able to react to changing requirements and to add new features in a reasonable time.
    - This may sound easy, but it is not, because it requires a strong commitment to invest into limiting technical debt. By its very definition, technical debt consists of trading off current for future velocity. Unfortunately, the amount to which future velocity has been compromised is often only apparent to the product side once it is already too late.
  - Maximizing business value by finding the right trade-off between investing into the future vs realizing current value. In financial terms, the value of an investment is the expected value of the discounted future cash flows (net present value/NPV).
  - Allow us to tackle problems of greater *inherent* complexity by limiting the *accidental* complexity of our system.
- Efficiency
- Align incentives: Unless it is in everyone's best interest to follow the best practices, any progress made is brittle.

## Implications
### Must-haves: 
- Implement quick wins, even if the benefits seem not essential: It follows from the principle to maximize NPV that it would be foolish to forgo small investments that deliver an outsized benefit - even if those benefits may seem like they are only nice to have rather than a must-have. Example: configuring code autoformatting in the CI pipeline.
- Investments, even if sizable, that prevent a lot of substantial problems down the road that may take up much the team's time. Examples: focusing on data quality first (versioning, how the productionize data/data transforms, etc); implementing proper logging and monitoring before moving to production, to avoid looking for the proverbial needle in the haystack once something goes wrong.

# *ML Engineering* best practices
## Must have
-Team:
  - Cross-functional: slice by value -> data scientists and ML engineers on same team
  - generalizing specialists, to balance: 
    - having multiple team members who can do a given job, in order to avoid bottlenecks and a single place of failure. (This is the "generalizing" part.)
    - since we are dealing with a domain that is both exceptionally large and complicated, we do require a good amount of specialized knowledge. This problem is especially pronounced in ML, because it touches on so many fields that it is hard to have all the necessary skills represented even on a reasonably sized (5-8) team: In addition to the core expertise in data/ML science and MLOps, we also need knowledge in software engineering, DevOps, and data engineering.
  - Since it is usually only realistic to get broad enough expertise represented on the team to help it solve *everyday* challenges (at least in fields beyond the team's core mission), it is vital that the team still has access to company-wide specialists for questions that are either hard or important (risk, long-term impact, etc.). Bad (or even sub- optimal) decisions can be very expensive, so it is crucial to make a small investment upfront to avoid these downstream problems as much as much as possible!

- Clean code: Prioritizing readability and maintainability
  - Refactor for readability, e.g.:
    - proper naming
    - separate levels of abstraction (e.g., a function should only do a single thing)
    - Don't do too much in a single line
      - Leverage design patterns to decouple application components
    - Reserve comments for the "why", not the "what" (which should be apparent from the code itself)
  - Follow coding standards. Rationale: See for example [the excellent discussion in "Software Engineering at Google"](https://abseil.io/resources/swe-book/html/ch08.html)
    - Pick an existing standard (e.g., PEP-8 or Google Python Style Guide)
    - Customize it, if desired. (Make sure to document the rationale for decisions.) 
    - Define as a configuration file (e.g., .pylintrc + mypy.ini). An easy way to get started is [the .pylintrc from Google's Style Guide](https://github.com/google/styleguide/blob/gh-pages/pylintrc)
    - Enforce by running linting in CI pipeline. 
    - Optionally, configure IDE to run linting on file save. Generally, it makes sense to at least run basic linting, such as fixing whitespace issues, in IDE.
  - Rationale for why clean code is a must-have:
    - Increase efficiency by aligning incentives: Code is read much more of than written -> it should be optimized for readability.
    - Reduces bugs.
    - Increasing Agility, because it makes code easier to change.
    - Overall, reduces maintenance cost (which is majority of the cost of a typical software project)
  - Note: Clean code should be part of our definition of done. If we made it a separate story, we would be using a "Scrum-fall"/"mini-waterfall". (Plus, we all know that it's too easy to indefinitely postpone these important but not urgent problems).


- Leverage design patterns to achieve loose coupling between components -> proper abstractions
  - Rationale:
    - Greatly reduces complexity, thereby ensuring code stays maintainable (reduces cost and risk, while increasing speed of feature implementation)
  - Software design patterns: 
    - The idea of design patterns is most closely associated with [the "gang of four's" seminal book](https://en.wikipedia.org/wiki/Design_Patterns), though I would most highly recommend ["head-first design patterns"](https://legacy.cs.indiana.edu/classes/c212-dgerman/spr2015/griffin/a.pdf) for a simpler and briefer explanation. 
    - [SOLID principles](https://en.wikipedia.org/wiki/SOLID)
  - Modern, cloud-native architecture: While I would only consider following the [Reactive principles](https://www.reactiveprinciples.org/cloud-native/index.html) a *should*-have, I do still think that leveraging some of its most basic insights is a *must*-have. In particular:
    - prefer *asynchronous* over synchronous communication;
    - create an *event-based* architecture (rather than triggering based on a schedule or synchronous API calls);
    - For example: 
      - To communicate to downstream service that the upstream service has finished, put a message in a queue (e.g., SQS), which in turn is configured as an [event source](https://docs.aws.amazon.com/lambda/latest/dg/with-sqs.html) for the downstream service, thereby invoking it and processing this message. 
      - In some cases, the outcome of the upstream processing can directly trigger the downstream processing: In partifular, [a file upload to a specific S3 bucket can invoke a Lambda function](https://docs.aws.amazon.com/lambda/latest/dg/with-s3.html). (E.g., every time new image has been uploaded to our "raw data" bucket, this triggers a lambda invocation that initiates downstream processing of that image, such as feature extraction.)
      - Overall, by contrast to using a synchronous API call, these examples avoid the need for both systems to be up and running at the same time, thereby decoupling their failures. This gives us not just higher availability, but also easier debugging and simpler reprocessing if needed.



- Trusted, automated test suite
  - Components:
    - unit tests
    - integration/acceptance tests
    - data validation
    - static analysis
  - Rationale why this is a must-have:
    - Increases quality (reduces errors, outages, etc.)
    - Decreases costs, since the cost of bugs rises the later in the SDLC they are discovered ("shifting left")
    - Indirect benefit: A trusted test suite makes sure that engineers are not dominated by their own creation
  - Rationale : Why do we still need unit tests, even if we have acceptance test that ensure that all of our features are working?
    - Makes debugging easier because 
      - Shortens the feedback loop (since running unit tests should be quick, so they can be run much more often);
      - If a bug is introduced, the fact *which specific* test fails shows us where exactly the problem is located (in which function or method);
    - gives us a safety harness that ensures it is safe to engage in refractoring, which in turn makes sure engineers won't soon be dominated by their own creation.



- Type safety:
  - code: use type hints, and run type checking (e.g. using mypy or pyre) 
    - Leverage the *gradual* nature of python's type system, which:
      - Makes it possible to apply type checking to only *part* of the code base, and increase coverage and strictness over time.
      - Does not require data scientists to add type annotations; but allows engineers to easily add these improvements later on.
    - Use the types in annotations: In particular, don't use dictionaries for heterogeneous collections. Often dataclasses/attrs or a custom class is the best choice here. Otherwise, the type checker has no way of knowing if a given attribute access or method call is valid or not.
    - *enforce* type checking in CI pipeline:
      - fail pipeline if type checks fail
      - guard against regression:
        - If codebase is fully typed, disallow untyped expressions or the use of "Any". (Of course, we do need to give the option to override this on a line by line basis. However, PR reviewers should enforce that the reason is documented in a comment, and that it is justified).
        - As long as we are still on the journey to full coverage, we need to at least make sure we are making progress by enforcing that each successive commit *increases* the overall type coverage.
  - data: use explicit data schemas
    - At a minimum, check schema at input and output.
    - Tools for semi-structured data:
      - For parsing JSON, such as an API response or configuration read from a file, pydantic is usually the best choice. It is very performant (v2 has been re-implemented in Rust) and offers a lot of useful configuration options, such as whether to allow type-casting. Thus, it has become the industry standard for this use case.
      - Where no parsing is necessary (i.e., when the data is already represented as Python objects), dataclasses and attrs have a performance advantage over pydantic.
    - Tools for tabular data:
      - Store data in a format that includes the schema (e.g., parquet).
      - Good libraries for defining the schemas for Pandas (and other) DataFrames that that allow us to define the datatypes of columns (and optionally the column order), as well as other essential column constraints such as uniqueness and nullability:
        - pandera: My favorite, because you define schema right in the code (like a type hint), thereby serving the additional function of documentation that enhances code readability.
        - the great expectations 
       - Note that it is also possible to enforce the data *types* when reading data into Pandas by passing the `dtypes` option, but this does not allow us to specify any other column constraints.
    - Common antipatterns:
      - Store data in CSV format: represents everything as text, so does not guarantee that data types will be preserved across read and writes, and has no representation for missing values; very slow to write and read; large file size.
      - *Infer* types: This is brittle because unexpected changes in the input data can lead to bugs or - even worse - semantic errors (e.g., if `False` is read as a string, it may evaluate to `True`, depending how the check is performed). Of course, infering types is often necessary during the interactive data science phase, were we manually inspect the output of each code cell, and then react to what we observe. However, this is not fit for production use, where we would like our code to keep working without requiring ongoing babysitting.
        - e.g., reading data into a dataframe without passing dtypes argument or otherwise enforcing schema;
        - e.g., reading a config from a file into a python dictionary without checking or casting the types of the keys and values. 
        - e.g., using AWS Glue Crawler to infer schema for tabular data, rather than defining it explicitly.


- Observability:
  - code
    - structure & centralized logging
    - monitoring and alerting
    - distributed tracing, if using micro-service architecture
  - model performance 
    - comparison between different models
    - comparison of same model over time (model drift?)
    - performance for different subsets of the population/bias (if substantial, does this vary by model?).
    - Note that model performance is not always observable right away. If it is not, model validation becomes much harder, and we should pick a variety of validation metrics to track. In this case, comparison is even more challenging, especially if there is no obvious way to boil down the different validation metrics to just a single metric.
      - A prominent example of this problem is the failure of Zillow's attempt to leverage their price prediction algorithm in order to buy underpriced properties and sell them at a premium: While it was straightforward to measure the performance of a model for predicting historical transaction prices, it was virtually impossible to know how the model performed in production until a few months later, when the properties had been turned around and sold to new buyers. Only then did it become apparent that the model did very poorly, at which point large financial losses had already occurred. (The problem was that the bids that Zillow won were not a random sample of all properties, but tended to be those where the model most over-estimated the value; thus, positive and negative prediction errors on the whole sample did not cancel out.)
  - data
    - data quality: 
      - metrics and alerting on schema validation failures.
      - metrics and visualizations of other data quality metrics


- DevOps
  - Use infrastructure-as-code
    - CI/CD
    - Enforce quality gates in pipeline
      - in particular:
        - tests: run unit and acceptance tests, check test coverage threshold, static code analysis (especially type checking)
        - readability: linting or auto-formatting, code complexity checks
        - security scans: scan dependencies for vulnerabilities and license risk scan, static analysis
      - Note: Enforcement requires that pipeline blocks deployment if any of these checks fail.


- Agile architecture: Defer important decisions until the *latest responsible moment* – but not any later!
  - Note that this entails a balance between, on the one hand, not prematurely worrying about details we can figure out later; but on the other hand, we do need to think ahead about the big picture when making architectural choices, so that we avoid getting trapped on an inferior path.
    - Example: If there is a business use case for stream processing, the actual *implementation* of stream processing is probably something that can wait. However, any architectural and technology choices should take into account that we need to be able to easily add this functionality later on, without having to undergo a complete redesign.


- Data infrastructure
  - Find out precise enough business requirements before picking storage solutions. E.g., precise latency requirements of final product. This sounds common-sense, but is usually not done, both because of:
    - understanding Agile wrong: While there are in fact many details we should *not* worry about until later, latency requirements are not one of them, because they dictate architectural choices required to achieve them. Thus, defering this decision may lock you into an inferior architecture that requires a complete redesign to overcome these constraints.
    - it is not straightforward to get these answers from the business stakeholders, because they likely won't know the answers until the technical side helps them think through these details.
  - Sufficient flexibility of table schemas: Schema evolution should be possible easily - unlike in the traditional data warehouse. Technological advancements  have made this not just possible but easy (e.g., Delte Lake, Apache Iceberg). Given how much easier it makes the lives of data scientists (as well as data and ML engineers), this should be a no-brainer.
  - Lineage / provenance
    - Provided by [feature store](./_details/why_feature_stores.md). Can also come from other data engineering tools (e.g., Hudi, Deltalake, Iceberg)  for any upstream transformations. However, this is usually handled by separate data engineering team that owns creation of "silver tables" etc., but is important for investigating data quality issues that arise at this earlier stage.
    - Rationale for why lineage is a must-have: 
      - Required for **reproducibility** of our results: Even if the code is versioned in GitHub etc., we save all configs (incl. hyperparameters) in an experiment tracker, and model arifacts in a model registry - this only tells half the story if we don't know what exactly the data looked like when the training code was run.
        - Reproducibility is even a regulatory requirement for some use cases;
        - Even where it is not, it makes *debugging* inferior model performance or model bias *much* easier, preventing the team from eventually spending most of it's time searching for the proverbial needle in the haystack.
        - Makes *remediating* data quality issues much easier, because it becomes easy to find out which model versions have been affected by an identified problem.
      - Thus, the only cases in which it may be justifiable to do without data lineage is:
        - very high data quality (which is rare);
        - the model is not yet used in production;
        - the model is not used for any high-stakes decisions AND we get quick feedback on model performance (e.g., can observe the true value soon after, allowing us to at least detect any performance problems quickly).

- Feature Engineering:
  - Lineage/provenance (see above)
  - Ensure same transformations are applied during training and inference (to avoid training-serving skew)
    - This is [another major reasons](./_details/why_feature_stores.md) to use a future store.
        


- Invest some time to find *good* (but not perfect) tools for the job
  - Avoid reinventing the wheel -> Leverage managed services wherever possible (unless don't find one with “fair” pricing for the use case)
  - leverage power of IDE (rather than notebook in browser)

## Should have

- Enable real-time intelligence (if there is a business use case):
  - Support stream processing
    - Depending on use case, micro-batch (e.g. Spark Structured Streaming) may be enough, or "real" (single-record) stream processing (e.g., Flink) may be necessary for lowest latency.
    - Note that this sometimes requires substantial changes to the data architecture, if it has not been designed with streaming in mind. Generally, we want to avoid doing all the work twice ("Lambda Architecture"), but instead leverage an architecture that allows both batch and streaming ("Kappa Architecture").
  - *Online* feature store
  - Low-latency inference and serving

- Identify and optimize performance bottlenecks
  - Profile code
  - Why we do not consider this a must-have: Engineers' time is very expensive, and so is delaying features or accumulating technical debt. Thus, performance optimization has sometimes to be sacrificed for these even more important goals.

- Periodically reevaluate if there are better tools for the job
  - Data processing
    - e.g., Pandas alternatives (such as Polars for greater speed and memory efficiency, or Modin + Ray for even greater speedup by parallelizing across multiple machines)
    - e.g., Spark vs Presto
  - MLOps tools
    - If started out with an end-to-end platform (e.g., Sagemaker): Is it worth switching to any of the best-off-breed MLOps tools for specific parts of the ML lifecycle (e.g., Tecton as feature platform, MLFlow for experiment tracking, Seldon for model serving, etc.)

- More advanced design patterns
  - Cloud-native architecture: 
    - [Reactive principles](https://www.reactiveprinciples.org/patterns/index.html), especially if using distributed system (e.g., container-orchestration)

- Observability:
  - Easy way to move between these three dimensions (e.g., if monitoring shows a spike in out-of-memory errors, we should be able to easily pull up the application logs from that particular machine.)
  - Distributed tracing, if the application contains more than a few cross-service calls (even if it is not a true micro-services architecture).
- DevOps:
  - Infra-as-Code:
    - Also manage most of the *data science* infrastructure using IaC. 
      - The reason I say "most" is because we get the biggest bang for our buck by focusing on the constant/long-lived infrastructure components; if a data scientist wants to try out some new resources, it's fine to create it using the console - thereby, we avoid introducing dependencies/blockers.
  - CI/CD:
    - Manual modifications to prod should only be possible in emergencies 
      - Engineers shoul have read-only access to prod
      - "break-glass account" (with sign-off process) for emergencies



## Would-like to have

## (Does not need to have:)

- Best practices for data science:
